### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS

#### For a given PMID, provides associated GEO dataset ID(s) (GSEXXX)

#### Protocol:

#### Register for a account in NCBI and copy the API KEY provided in NCBI account > NCBI Account Settings
#### Provide a file with list of publications, just numbers no prefix. 

#### NOTE:
#### Needs Biopython module
#### Entrez allows max 10 requests per second (If API key is provided) otherwise the limit is 3 requests/second. 

In [ ]:
from Bio import Entrez
import time

In [ ]:
# Set up NCBI creds

Entrez.email = ""  # Email associated with your NCBI account
Entrez.api_key = ""  # API key can be copied from your NCBI account > NCBI Account Settings


In [ ]:
def get_gse(pmid_accession_list):
    no_gse = False
    
    for pmid in pmid_accession_list:
        handle_gds_uid = Entrez.elink(dbfrom="pubmed", id=pmid, db="gds") #get GDS UID (which is different from GSE accession)
        record_gds_uid = Entrez.read(handle_gds_uid)
        handle_gds_uid.close()
        if len(record_gds_uid[0]["LinkSetDb"]) == 0: #If there is no GEO datasets (GDS) with the PMID
            no_gse = True
            gse_accession = "Not (yet) in GEO"
        elif len(record_gds_uid[0]['LinkSetDb'][0]['Link']) >= 1: #If PMID is associated with GEO Datasets (GDS)
            gds_uids = []
            list_gds_acc = record_gds_uid[0]['LinkSetDb'][0]['Link']
            for gse_acc in list_gds_acc:
                gds_uids.append(gse_acc['Id'])
            for ids in gds_uids:
                handle2_gse_acc = Entrez.esummary(db="gds", id=ids)  # convert GDS UID to GSE accession
                record2_gse_acc = Entrez.read(handle2_gse_acc)
                handle2_gse_acc.close()
                gse_accession = record2_gse_acc[0]['Accession']
                print(pmid,gse_accession)
                time.sleep(5) 
    
        if no_gse:
            print(pmid, gse_accession)
            time.sleep(7)

In [ ]:
# File with a list of PMIDs - one on each line.

input_file = ''
chunks = 5       #Splits the PMID list into specified no. of values per list

with open(input_file) as file1:

    all_pmids = []
    for pmid in file1:
        pmid = pmid.rstrip()
        all_pmids.append(str(pmid))

for count in range(0,len(all_pmids),chunks):
    pmid_accession_list = all_pmids[count:count+chunks]
    get_gse(pmid_accession_list)
    time.sleep(5)
    